# Sliding Window Tool
So the idea is that we take an input `img` and make a list of coordinates (`coords`) that we're going to take a crop at (with dimensions `win_dims`).

So in this next cell we import `img` and generate the list `coords` based on the number of `grid_slices` and `sub_slices` we have. `grid_slices` indicate the size of the window and the `sub_slices` indicate how many times that slice should be taken. For example, `grid_slices = 2` and `sub_slices = 0` would just give you 4 images with non-overlapping 2x2 cropping. In contrast `grid_slices = 2` and `sub_slices = 2` would give you 9 images. 
```
|--*--|-----|
|     |     |
|     |     |
*  *  *     |
|     |     |
|     |     |
|--*--|-----|
|     |     |
|     |     |
|     |     |
|     |     |
|     |     |
|-----|-----|
```
In this graphic, the stars indicate the upper left corner of overlapping crops (designated by `sub_slice`) and the `-` and `|` indicate the `grid_slices`.

Writing it out, I see this is a crazy way to do this, sorry

<sup>Author: Jamie Eckstein jamie.k.eckstein@gmail.com</sup>

In [3]:
import numpy as np
from skimage import io
import os
from src.crop import crop
from src.generate_coords import generate_coords

# Cropping
This crops images into overlapping sub images

In [4]:
# this is the stuff you need to change (like filenames and stuff) and generate coords

img = io.imread("data/01.png")
img = img[:,:,0]
out_dir = "data/croppings/"
fname_ind = 0 # This is the index of the first image you want to output

grid_slices = 2
sub_slices = 2

coords = generate_coords(grid_slices, sub_slices, img.shape[:2])

In [5]:
# Iterate over coords and save crops
img_dims = img.shape[:2]
win_dims = (img_dims[0] // grid_slices, img_dims[1] // grid_slices)

for ind, pt in enumerate(coords):
    cropped_img = crop(img, pt, win_dims)
    if cropped_img is not None:
        io.imsave(f"{out_dir}{ind + fname_ind}.png", cropped_img)

# Compiling

In [7]:
# This is the stuff you need to change (like filenames and stuff) and generate coords

out_fname = "compilation.png"
in_dir = "data/croppings/"
fname_ind = 0 # This is the index of the first image you want to compile
fname_suf = ".png"

grid_slices = 2
sub_slices = 2

img = io.imread(f"{in_dir}{fname_ind}{fname_suf}")
if len(img.shape) > 2:
    img = img[:,:,0]

coords = generate_coords(grid_slices, sub_slices, win_dims = img.shape[:2])

In [8]:
win_dims = img.shape[:2]
width = win_dims[0] * grid_slices
height = win_dims[1] * grid_slices

compiled_img = np.zeros((width, height))

for i, pt in enumerate(coords):
    fname = f"{in_dir}{fname_ind + i}{fname_suf}"
    print(f"Reading: {fname}")
    sub_img = io.imread(fname)
    if len(sub_img.shape) > 2:
        sub_img = sub_img[:,:,0]
    compiled_img[pt[0]:pt[0] + win_dims[0],pt[1]:pt[1] + win_dims[1]] += sub_img

# average normalizing
compiled_img[win_dims[0] // 2:-win_dims[0] // 2,:] /= 2
compiled_img[:,win_dims[1] // 2:-win_dims[1] // 2] /= 2
print(f"Writing to: {out_fname}")
io.imsave(out_fname, compiled_img.astype(np.uint8))

Reading: data/croppings/0.png
Reading: data/croppings/1.png
Reading: data/croppings/2.png
Reading: data/croppings/3.png
Reading: data/croppings/4.png
Reading: data/croppings/5.png
Reading: data/croppings/6.png
Reading: data/croppings/7.png
Reading: data/croppings/8.png
Writing to: compilation.png
